## Import modules

In [ ]:
!pip install mxnet-mkl==1.6.0 numpy==1.23.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 28.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.1 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.23.1 which is incompatible.
tensorflow 2.15.0 requires numpy<2.0.0,>=1.23.5, but you have numpy 1.23.1 which is incompatible.


In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-2hedbphz/kobert-tokenizer_126d79d2c402485b93728f4e5b805e73
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-2hedbphz/kobert-tokenizer_126d79d2c402485b93728f4e5b805e73
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=5512b14f536316d400921424a4f75505c749af47e51bc2c014042204bd4b4932
  Stored in directory: /tmp/pip-ephem-wheel-cache-r_6eeorh/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
!pip install gluonnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661755 sha256=f7038b5de2217b6ecc050ac09ce86dd06a75b7906469414147df4a3cbdeefb4f
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp


**Requirement**

* Python >= 3.6
* PyTorch >= 1.8.1
* transformers >= 4.8.2
* sentencepiece >= 0.1.91

In [ ]:
#혹시 또 nlp에서 boolean에 대해 error가 생기면 처리할 코드
#np.bool = np.bool_

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from sklearn.model_selection import train_test_split

import os

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0")
# device = torch.device("cpu")

### Pretrained KoBERT model and tokenizer

In [ ]:
#KoBERT Pretrained Tokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
#KoBERT Pretrained Model
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1')

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
#모델과 tokenizer import가 잘 되었는지 확인할 수 있는 코드
'''
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1')
text = '[CLS] 한국어 모델을 공유합니다. [SEP]'
inputs = tokenizer.batch_encode_plus([text])
out = bertmodel(input_ids = torch.tensor(inputs['input_ids']),
            attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape
'''

torch.Size([1, 768])

In [ ]:
#nlp module의 BERTVocab을 사용할 vocab으로 지정
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

##DATA Preprocessing

In [ ]:
df1 = pd.read_csv('KDI 시사용어사전.csv')
df1

,용어,설명
0,가격 차별,"소비자의 나이나 신분, 재화가 판매되는 공간의 지리적 요인에 따라 가격을 다르게 책..."
1,가격 차별,기업은 가격을 차별함으로써 이윤을 극대화하기도 한다
2,가격 차별,일상에서 찾아볼 수 있는 가격차별의 예시로는 나이에 따라 다르게 매겨지는 대중교통 ...
3,가격 차별,또한 동일한 상품이라도 국내에서 출시하는 상품과 해외에서 출시하는 상품의 가격을 다...
4,가격 차별,"가격차별이 가능하려면 첫째, 소비자를 몇 개의 그룹으로 분류할 수 있어야 하고, 둘..."
...,...,...
7766,히든챔피언,히든챔피언 기업의 공통된 특성은 다음과 같다
7767,히든챔피언,먼저 한 분야의 전문가로 시장을 좁게 정의하고 있으며 세계화에 공을 들인다
7768,히든챔피언,또 아웃소싱을 하되 연구개발(R&D) 등 핵심역량은 직접 수행한다
7769,히든챔피언,"고객 친밀성이 높아 VIP 고객들과 밀접한 관계를 구축하고, 기업문화는 직원에게 일..."


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7771 entries, 0 to 7770
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   용어      7771 non-null   object
 1   설명      7451 non-null   object
dtypes: object(2)
memory usage: 121.5+ KB


In [ ]:
#데이터 결측치 제거
df1 = df1.dropna()

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7451 entries, 0 to 7770
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   용어      7451 non-null   object
 1   설명      7451 non-null   object
dtypes: object(2)
memory usage: 174.6+ KB


In [ ]:
#용어에 대한 설명이 10개 이상인 용어만 사용
count = df1['용어'].value_counts()
over10 = list(count[count >= 10].index)

In [ ]:
#학습할 용어
over10

['신용평가',
 '쟁의행위',
 '조세피난처',
 '독점적 경쟁시장',
 '듀레이션',
 '배당 ',
 '인공지능',
 '프로젝트사이클 ',
 '영세율',
 '사전심사청구제도',
 '넉인/넉아웃',
 '예산과목',
 '민간투자사업',
 '개인파산제도',
 '배출권거래제',
 '부동산투자신탁회사',
 '자산재평가',
 '적기시정조치',
 '컨소시엄',
 '카이젠',
 '워크아웃',
 '워런트',
 '양도성예금증서',
 '서비스거래에 관한 일반협정',
 '예산',
 '환경라벨링',
 '예금보험제도',
 '비트코인',
 '생애 주기/항상 소득 가설',
 '주택분양가원가연동제',
 '구조적 실업',
 '상시구조조정',
 '죄수의 딜레마',
 '자산부채이전',
 '호텔링 모형',
 '연방준비제도이사회',
 '액면분할',
 '원산지규정',
 '법정관리',
 '승수효과',
 '절대적 빈곤',
 '세계무역기구',
 '수권자본제도',
 '수요의 교차 가격탄력성',
 '신용 창조',
 '수요의 법칙',
 '신경제',
 '전세권 설정',
 '구조조정',
 '잉여현금흐름',
 '시장경제지위',
 '기업결합',
 '미주개발은행',
 '별정통신사업자',
 '황의법칙',
 '동의명령제',
 '가치의 역설',
 '디레버리징',
 '근원 인플레이션',
 '무역관련 지적재산권에 관한 협정',
 '발생주의회계ㆍ현금주의회계',
 '테크노크라시',
 '국부펀드',
 '상가건물임대차보호법',
 '부가가치세',
 '결산',
 '스위트 스폿',
 '사내 벤처',
 '기준금리',
 '기축통화',
 '남북청산결제',
 '금융허브',
 '대역폭',
 '앱카드',
 '연평균복합성장률',
 '국세',
 '블루칩',
 '변액보험',
 '신자산건전성 분류기준',
 '분식회계',
 '미수거래',
 '미스터리 쇼핑',
 '신BIS협약',
 '상장지수펀드',
 '그룹웨어',
 '밸류에이션',
 '선물거래',
 '역외선물환',
 '경로 의존성',
 '유한회사/합자회사',
 '경기순환',
 '추곡수매제도',
 '오페라본드

In [ ]:
df = df1[df1['용어'].isin(over10)]
df

,용어,설명
36,가산금리,기준금리에 신용도 등의 조건에 따라 덧붙이는 금리를 가산금리(스프레드: spread...
37,가산금리,"즉, 대출 등의 금리를 정할 때 기준금리에 덧붙이는 위험가중 금리를 말한다"
38,가산금리,"따라서 신용도가 높아 위험이 적으면 가산금리가 낮아지고, 반대로 신용도가 낮아 위험..."
39,가산금리,실제로 우리나라는 1997년 외환위기시 부족했던 달러화를 해외에서 차입하거나 외평채...
40,가산금리,이는 우리나라가 외환위기로 인해 대외신인도가 그만큼 낮아진데 따른 것이다
...,...,...
7683,황의법칙,그 내용은 반도체의 집적도가 2배로 증가하는 시간이 1년으로 단축되었으며 무어의 법...
7684,황의법칙,그리고 이를 주도하는 것은 모바일 기기와 디지털 가전제품 등 non-PC분야라고 하였다
7685,황의법칙,이 규칙을 황창규 사장의 성을 따서 ‘황의 법칙’이라고 한다
7686,황의법칙,"실제 삼성전자는 1999년에 256M 낸드플래시메모리를 개발하고, 2000년 512..."


In [ ]:
label_list = list(df['용어'].unique())
print(len(label_list))

#one-hot encoding을 위한 mapping
label_mapping = {}
for idx,term in enumerate(label_list):
  label_mapping[term] = idx

108


In [ ]:
#output 도출을 위한 dict
result_dict = {}
for idx,term in enumerate(label_list):
  result_dict[idx] = term

In [ ]:
df #최종적으로 사용할 데이터

,용어,설명
36,가산금리,기준금리에 신용도 등의 조건에 따라 덧붙이는 금리를 가산금리(스프레드: spread...
37,가산금리,"즉, 대출 등의 금리를 정할 때 기준금리에 덧붙이는 위험가중 금리를 말한다"
38,가산금리,"따라서 신용도가 높아 위험이 적으면 가산금리가 낮아지고, 반대로 신용도가 낮아 위험..."
39,가산금리,실제로 우리나라는 1997년 외환위기시 부족했던 달러화를 해외에서 차입하거나 외평채...
40,가산금리,이는 우리나라가 외환위기로 인해 대외신인도가 그만큼 낮아진데 따른 것이다
...,...,...
7683,황의법칙,그 내용은 반도체의 집적도가 2배로 증가하는 시간이 1년으로 단축되었으며 무어의 법...
7684,황의법칙,그리고 이를 주도하는 것은 모바일 기기와 디지털 가전제품 등 non-PC분야라고 하였다
7685,황의법칙,이 규칙을 황창규 사장의 성을 따서 ‘황의 법칙’이라고 한다
7686,황의법칙,"실제 삼성전자는 1999년에 256M 낸드플래시메모리를 개발하고, 2000년 512..."


In [ ]:
label_mapping

{'가산금리': 0,
 '가치의 역설': 1,
 '개인파산제도': 2,
 '결산': 3,
 '경기순환': 4,
 '경로 의존성': 5,
 '구조적 실업': 6,
 '구조조정': 7,
 '국부펀드': 8,
 '국세': 9,
 '그룹웨어': 10,
 '근원 인플레이션': 11,
 '금융허브': 12,
 '기업결합': 13,
 '기준금리': 14,
 '기축통화': 15,
 '남북청산결제': 16,
 '넉인/넉아웃': 17,
 '대역폭': 18,
 '독점적 경쟁시장': 19,
 '동의명령제': 20,
 '듀레이션': 21,
 '디레버리징': 22,
 '무역관련 지적재산권에 관한 협정': 23,
 '미수거래': 24,
 '미스터리 쇼핑': 25,
 '미주개발은행': 26,
 '민간투자사업': 27,
 '발생주의회계ㆍ현금주의회계': 28,
 '배당 ': 29,
 '배출권거래제': 30,
 '밸류에이션': 31,
 '법정관리': 32,
 '변액보험': 33,
 '별정통신사업자': 34,
 '부가가치세': 35,
 '부동산투자신탁회사': 36,
 '분식회계': 37,
 '블루칩': 38,
 '비트코인': 39,
 '사내 벤처': 40,
 '사전심사청구제도': 41,
 '상가건물임대차보호법': 42,
 '상시구조조정': 43,
 '상장지수펀드': 44,
 '생애 주기/항상 소득 가설': 45,
 '서비스거래에 관한 일반협정': 46,
 '선물거래': 47,
 '세계무역기구': 48,
 '수권자본제도': 49,
 '수요의 교차 가격탄력성': 50,
 '수요의 법칙': 51,
 '스위트 스폿': 52,
 '승수효과': 53,
 '시장경제지위': 54,
 '신BIS협약': 55,
 '신경제': 56,
 '신용 창조': 57,
 '신용평가': 58,
 '신자산건전성 분류기준': 59,
 '액면분할': 60,
 '앱카드': 61,
 '양도성예금증서': 62,
 '역외선물환': 63,
 '연방준비제도이사회': 64,
 '연평균복합성장률': 65,
 '영세율': 66,
 '예금보험제도':

##Model

###Dataset

In [ ]:
train, test = train_test_split(df, test_size = 0.2, shuffle = True, random_state = 42, stratify = df['용어'])

In [ ]:
#Dataset 자료 형식 : [[용어1, 설명1], [용어1, 설명2], ... , [용어2, 설명1], ...] 형식의 리스트
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
#BERTDataset에 넣기 적절한 모양으로 바꿈
dataset_train = []
dataset_test = []

lab = list(train.iloc[:,0])
sen = list(train.iloc[:,1])

for l,s in zip(lab, sen):
  dataset_train.append([label_mapping[l],s])

lab2 = list(test.iloc[:,0])
sen2 = list(test.iloc[:,1])

for l,s in zip(lab2, sen2):
  dataset_test.append([label_mapping[l],s])

In [ ]:
tok = tokenizer.tokenize

In [ ]:
data_train = BERTDataset(dataset_train, 1, 0, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 1, 0, tok, vocab, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


###Bert Model

In [ ]:
#hyperparameter
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5
numclasses = len(label_list)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=numclasses,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(),
                              attention_mask = attention_mask.float().to(token_ids.device),
                              return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

##Training

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.1).to(device)

In [ ]:
#optimization options
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
#train
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train() #model을 train 모드로 설정
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-90-1e396f699656>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/32 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch 1 batch id 1 loss 4.730868816375732 train acc 0.0
epoch 1 train acc 0.0126953125


<ipython-input-90-1e396f699656>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

epoch 14 test acc 0.7653245192307693


  0%|          | 0/32 [00:00<?, ?it/s]

epoch 15 batch id 1 loss 1.7081513404846191 train acc 1.0
epoch 15 train acc 0.984375


  0%|          | 0/8 [00:00<?, ?it/s]

epoch 15 test acc 0.7689302884615384


## Test

In [ ]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

#train된 모델을 통해 output을 확인할 때 사용
def testModel(model, seq):
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, vocab, pad=True, pair=False)
    tokenized = transform(tmp)

    model.eval()
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    print("입력 내용의 연관 경제 용어는 \'"+ result_dict[idx] + '\'입니다.')
    #print("신뢰도는:", "{:.2f}%".format(softmax(result,idx)))


### Experiment

In [ ]:
testModel(model, "경쟁을 하는데 시장에서 독점적인 위치를 가지고 있습니다.")

입력 내용의 연관 경제 용어는 '독점적 경쟁시장'입니다.


In [ ]:
testModel(model, "주식회사에서 주주들에게 주식 수만큼 일정 금액을 돌려주는 것입니다.")

입력 내용의 연관 경제 용어는 '배당 '입니다.


In [ ]:
testModel(model, '국가에 돈이 부족해')

입력 내용의 연관 경제 용어는 '절대적 빈곤'입니다.
